In [ ]:
# Importa o módulo para acessar o Google Drive no Colab
from google.colab import drive

# Monta o Google Drive no ambiente do Colab,
# permitindo acessar arquivos como se estivessem em uma pasta local.
# O parâmetro 'force_remount=True' garante que o Drive será montado novamente,
# mesmo que já esteja montado anteriormente (útil para evitar problemas de cache).
drive.mount('/content/drive', force_remount=True)

# 📦 Entrega 2 – Comparação de Abordagens de Visão Computacional
Nesta etapa, comparamos três abordagens para reconhecimento de objetos utilizando o mesmo dataset:

- YOLOv5 customizado (adaptado na Entrega 1)
- YOLO tradicional (baseado no Capítulo 3 de Redes Neurais)
- CNN simples treinada do zero para classificação de imagens

A seguir, apresentamos os códigos, resultados e avaliações críticas de cada abordagem.

## 🔍 Abordagem 1: YOLOv5 Customizado

In [ ]:
# Exemplo de detecção com YOLOv5
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /content/drive/MyDrive/Dataset/A/test/images

In [ ]:
# Exibe imagens com detecções
import os
from IPython.display import Image, display
detect_path = '/content/yolov5/runs/detect/exp'
for img_name in os.listdir(detect_path):
    if img_name.endswith('.jpg'):
        display(Image(filename=os.path.join(detect_path, img_name)))

### 📝 Avaliação da YOLOv5 Customizada
- **Precisão**: Alta (ex: `mAP@0.5 = XX%`)
- **Tempo de treinamento**: Médio (~5-10 min)
- **Facilidade de uso**: Boa (documentação robusta)
- **Tempo de inferência**: Rápido
- **Comentários**: Ótima performance em detecção de objetos com rotulagem manual.

## 📘 Abordagem 2: YOLO Tradicional (Capítulo 3)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Caminhos para os dados organizados por classe (como classificação)
train_dir = '/content/drive/MyDrive/Dataset/A/train'
val_dir   = '/content/drive/MyDrive/Dataset/A/val'

# Pré-processamento das imagens
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=16, class_mode='categorical')
val_gen = datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=16, class_mode='categorical')

# Modelo YOLO simplificado (inspirado na arquitetura tradicional)
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')  # 2 classes: trator e plantacao
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinamento
model.fit(train_gen, validation_data=val_gen, epochs=10)

In [ ]:
loss, acc = model.evaluate(val_gen)
print(f'Acurácia da YOLO tradicional simulada: {acc:.2%}')

### 📝 Avaliação da YOLO Tradicional
- **Precisão**: Média ou Baixa (ex: `60%`)
- **Tempo de treinamento**: Baixo ou inexistente
- **Facilidade de uso**: Média
- **Tempo de inferência**: Lento
- **Comentários**: Modelo didático, mas pouco preciso em contextos reais.

## 🧠 Abordagem 3: CNN do Zero (Classificação de Imagens)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Diretórios
train_dir = '/content/drive/MyDrive/Dataset/A/train'
val_dir = '/content/drive/MyDrive/Dataset/A/val'
test_dir = '/content/drive/MyDrive/Dataset/A/test'

# Geradores de imagem (normalizando)
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=16, class_mode='categorical')
val_gen = datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=16, class_mode='categorical')
test_gen = datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=1, class_mode='categorical', shuffle=False)

# Modelo CNN simples do zero
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')  # 2 classes
])

# Compilação
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento
model.fit(train_gen, validation_data=val_gen, epochs=10)

In [ ]:
loss, accuracy = model.evaluate(test_gen)
print(f"Acurácia no conjunto de teste: {accuracy:.2%}")

### 📝 Avaliação da CNN
- **Precisão**: Média (ex: `80%`)
- **Tempo de treinamento**: Rápido
- **Facilidade de uso**: Alta
- **Tempo de inferência**: Rápido
- **Comentários**: Boa alternativa para classificação simples, mas não detecta localização dos objetos.

## 📊 Comparativo Final
| Abordagem         | Facilidade de uso | Precisão | Tempo de treino | Tempo de inferência |
|-------------------|-------------------|----------|------------------|----------------------|
| YOLOv5 Customizado| Alta              | Alta     | Médio            | Rápido               |
| YOLO Tradicional  | Média             | Média    | Baixo            | Lento                |
| CNN do Zero       | Alta              | Média    | Rápido           | Rápido               |

## ✅ Conclusão
Após testar as três abordagens, ficou evidente que a **YOLOv5 customizada** apresenta os melhores resultados em termos de precisão e aplicabilidade em problemas reais de detecção.

A **YOLO tradicional**, apesar de útil para fins educacionais, não é tão eficaz na prática. Já a **CNN do zero** é uma excelente alternativa para tarefas de **classificação**, mas não serve para detecção de múltiplos objetos em uma imagem.

**Cada abordagem tem seus pontos fortes, e a escolha depende do cenário específico.**